In [1]:


import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import glob
import os
import csv
from scipy.stats import f_oneway
from scipy.stats import shapiro, levene, probplot
import math
import matplotlib.pyplot as plt
from statsmodels.stats.multicomp import pairwise_tukeyhsd


def load_tensile_csv(csv_path):
    """
    Loads tensile raw data CSVs that may contain:
      - metadata header lines
      - mixed delimiters
      - inconsistent column counts
    Returns a cleaned DataFrame with numeric columns only.
    """

    # First: Detect delimiter
    with open(csv_path, "r") as f:
        sample = f.read(2048)

    try:
        sniffer = csv.Sniffer()
        dialect = sniffer.sniff(sample, delimiters=[',',';','\t',' '])
        delimiter = dialect.delimiter
    except:
        delimiter = ','  # fallback

    # Second: Read the file while skipping bad rows
    df = pd.read_csv(
        csv_path,
        sep=delimiter,
        engine="python",
        comment='#',
        skiprows=5,
        #on_bad_lines='skip',    # <-- prevents crash
        skip_blank_lines=True
    )

    # Third: Drop EMPTY or non-numeric columns
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        except:
            df[col] = pd.NA

    df = df.dropna(axis=1, how='all')  # remove columns with no data
    df = df.dropna(axis=0, how='any')  # remove unfinished rows

    print(df)
    return df

def compute_youngs_modulus(
    csv_path,
    gauge_length,
    cross_section_area,
    load_col='Load',
    disp_col='Extension',
    elastic_strain_max=0.005
):
    """
    Computes Young's modulus from tensile test data.

    Parameters
    ----------
    csv_path : str
        Path to CSV file containing load/displacement data.
    gauge_length : float
        Original specimen length (m).
    cross_section_area : float
        Cross-sectional area (m^2).
    load_col : str
        Column name for load/force.
    disp_col : str
        Column name for displacement/extension.
    elastic_strain_max : float
        Maximum strain to use for the linear elastic fit (default = 0.5%).

    Returns
    -------
    E : float
        Estimated Young’s modulus (Pa).
    stress : ndarray
    strain : ndarray
        Arrays of engineering stress and strain.
    """

    # Load file
    df = load_tensile_csv(csv_path)

    # Extract data
    force = df[load_col].values
    disp  = df[disp_col].values

    # Engineering strain
    strain = disp / gauge_length

    # Engineering stress
    stress = force / cross_section_area

    # Select elastic region
    mask = strain <= elastic_strain_max
    strain_elastic = strain[mask].reshape(-1, 1)
    stress_elastic = stress[mask]

    # Linear regression (slope = E)
    model = LinearRegression().fit(strain_elastic, stress_elastic)
    E = model.coef_[0]

    return E, stress, strain


In [2]:
def ultimate_tensile_strength(stress):
    """
    Maximum stress achieved in the test.
    """
    return np.max(stress)

def offset_yield_strength(E, strain, stress, offset=0.02):
    """
    Computes the offset yield strength (default 2%).
    Uses interpolation to find intersection of stress-strain curve with
    offset line of slope equal to the elastic modulus.
    """
    offset_line = E*(strain - offset)

    # difference between actual curve and offset line
    diff = stress - offset_line
    
    # find zero crossing
    idx = np.where(np.diff(np.sign(diff)))[0]
    if len(idx) == 0:
        return None  # No yield point found
    
    i = idx[0]
    # linear interpolation for better accuracy
    x1, x2 = strain[i], strain[i+1]
    y1, y2 = diff[i], diff[i+1]
    
    strain_yield = x1 - y1*(x2 - x1)/(y2 - y1)
    stress_yield = np.interp(strain_yield, strain, stress)
    
    return stress_yield

def total_energy(strain, stress):
    """
    Integrates the stress-strain curve using the trapezoidal rule.
    Units depend on stress/strain units (e.g., MPa -> MJ/m^3).
    """
    return np.trapz(stress, strain)

def strain_at_fracture(strain):
    """
    Maximum strain recorded before failure.
    """
    return np.max(strain)

In [3]:
def analyze_tensile_folder(folder_path, gauge_length, cross_section_area):
    """
    Processes all CSV tensile data files in a folder and returns a summary table.
    """

    results = []

    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
    # print(csv_files)

    for csv_path in csv_files:
        try:
            # --- Stress & strain + Young's modulus ---
            E, stress, strain = compute_youngs_modulus(
                csv_path,
                gauge_length,
                cross_section_area
            )
            # print("fail 1")

            # --- Individual metrics ---
            uts = ultimate_tensile_strength(stress)
            # print("fail1.5")
            #ys  = offset_yield_strength(strain, stress, E)
            # print("fail1.5")
            energy = total_energy(strain, stress)
            # print("fail1.5")

            eps_f = strain_at_fracture(strain)
            # print("fail 2")
            results.append({
                "Filename": os.path.basename(csv_path),
                "Youngs_Modulus_Pa": E,
                "UTS_Pa": uts,
            #    "YieldStrength_2pct_Pa": ys,
                "TotalEnergy_Jperm3": energy,
                "StrainAtFracture": eps_f
            })

            # print(results, uts, energy, eps_f)

        except Exception as e:
            print(f"Error processing {csv_path}: {e}")

    return pd.DataFrame(results)


In [4]:
reference_tests = analyze_tensile_folder(
     "./Reference Tests/Reference_T1",
    1, 1
)

print(reference_tests)


       Time  Extension         Load
1     0.000   -0.04256      1.60519
2     0.100   -0.03807      1.36151
3     0.200   -0.02903     12.88276
4     0.300   -0.01960    149.19690
5     0.400   -0.01041    256.47833
..      ...        ...          ...
717  71.062    5.87952  11136.10156
718  71.064    5.87972  10348.32031
719  71.066    5.87992   9540.51465
720  71.068    5.88010   8732.13672
721  71.070    5.88030   7930.69141

[721 rows x 3 columns]
       Time  Extension         Load
1     0.000    0.00000      2.46954
2     0.100    0.00403      4.91734
3     0.200    0.01346     88.88831
4     0.300    0.02311    184.77623
5     0.400    0.03238    305.11151
..      ...        ...          ...
739  73.342    6.11218  10917.00684
740  73.344    6.11238  10172.36230
741  73.346    6.11258   9401.66895
742  73.348    6.11278   8625.38672
743  73.350    6.11298   7852.75635

[743 rows x 3 columns]
       Time  Extension         Load
1     0.000   -0.08265     13.19987
2     0.100   -0

/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_82947/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_82947/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_82947/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_82947/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stre

In [5]:
#ANOVA TEST
def run_anova_on_metric(results_dict, metric_col):
    """
    results_dict: dict[name -> DataFrame from analyze_tensile_folder]
    metric_col: string name of the column to test, e.g. "UTS_Pa"
    """
    group_values = []
    group_labels = []

    for condition_name, df in results_dict.items():
        # pull that column, drop NaN
        vals = df[metric_col].dropna().values

        # we need at least 2 samples for ANOVA to make sense
        if len(vals) >= 2:
            group_values.append(vals)
            group_labels.append(condition_name)
        else:
            print(f"Skipping {condition_name} for {metric_col} (only {len(vals)} sample(s))")

    if len(group_values) < 2:
        raise ValueError(f"Need at least 2 groups with data for ANOVA on {metric_col}")

    F, p = f_oneway(*group_values)

    print(f"\nANOVA for {metric_col}")
    print("Groups:", group_labels)
    print(f"F-statistic = {F:.4f}, p-value = {p:.4e}")

    return F, p, group_labels

In [6]:
#Shapiro test
def run_shapiro_per_group(results_by_condition, metric_col):
    """
    Run Shapiro–Wilk test for normality in each group (condition) for one metric.
    """
    print(f"\n=== Shapiro–Wilk normality test for {metric_col} ===")
    shapiro_results = []

    for condition_name, df in results_by_condition.items():
        vals = df[metric_col].dropna().values

        # Shapiro needs at least 3 points to be meaningful
        if len(vals) < 3:
            print(f"{condition_name}: n={len(vals)} (skipping, too few samples)")
            continue

        stat, p = shapiro(vals)
        shapiro_results.append({
            "Condition": condition_name,
            "n": len(vals),
            "W_stat": stat,
            "p_value": p
        })

        interpretation = "✅ approx. normal (fail to reject H0)" if p > 0.05 else "⚠️ non-normal (reject H0)"
        print(f"{condition_name}: n={len(vals)}, W={stat:.4f}, p={p:.4f} → {interpretation}")

    return pd.DataFrame(shapiro_results)


In [7]:
#Q-Q plots

def qq_plots_per_group(results_by_condition, metric_col):
    """
    Generate Q–Q plots for each group (condition) for one metric.
    """
    n_groups = len(results_by_condition)
    ncols = 3  # adjust if you want a different layout
    nrows = math.ceil(n_groups / ncols)

    fig, axes = plt.subplots(nrows, ncols, figsize=(4 * ncols, 4 * nrows))
    axes = np.array(axes).reshape(-1)  # flatten in case of 2D

    fig.suptitle(f"Q–Q plots for {metric_col}", fontsize=16)

    for ax, (condition_name, df) in zip(axes, results_by_condition.items()):
        vals = df[metric_col].dropna().values

        if len(vals) < 3:
            ax.set_title(f"{condition_name}\n(n={len(vals)}, skipped)")
            ax.axis("off")
            continue

        probplot(vals, dist="norm", plot=ax)
        ax.set_title(f"{condition_name} (n={len(vals)})")

    # Hide any extra axes if number of groups < nrows*ncols
    for j in range(len(results_by_condition), len(axes)):
        axes[j].axis("off")

    plt.tight_layout()
    plt.show()


In [8]:
#LEVENES test
def run_levene_across_groups(results_by_condition, metric_col, center='median'):
    """
    Run Levene’s test for homogeneity of variances across groups for one metric.
    center: 'median' (more robust) or 'mean'
    """
    group_values = []
    group_labels = []

    for condition_name, df in results_by_condition.items():
        vals = df[metric_col].dropna().values
        if len(vals) < 2:
            print(f"{condition_name}: n={len(vals)} (skipping in Levene, too few samples)")
            continue
        group_values.append(vals)
        group_labels.append(condition_name)

    if len(group_values) < 2:
        raise ValueError(f"Need at least 2 groups with data for Levene’s test on {metric_col}")

    stat, p = levene(*group_values, center=center)

    interpretation = "✅ variances approx. equal (fail to reject H0)" if p > 0.05 else "⚠️ variances differ (reject H0)"

    print(f"\n=== Levene’s test for {metric_col} (center={center}) ===")
    print("Groups:", group_labels)
    print(f"Levene stat = {stat:.4f}, p = {p:.4f} → {interpretation}")

    return stat, p, group_labels


In [9]:
#Tukey HSD
def run_tukey_for_metric(results_by_condition, metric_col, alpha=0.05):
    """
    results_by_condition: dict[condition_name -> DataFrame from analyze_tensile_folder]
    metric_col: e.g. "UTS_Pa", "TotalEnergy_Jperm3", "StrainAtFracture"
    """
    all_values = []
    all_groups = []

    for condition_name, df in results_by_condition.items():
        vals = df[metric_col].dropna().values
        if len(vals) == 0:
            continue

        all_values.extend(vals)
        all_groups.extend([condition_name] * len(vals))

    all_values = np.array(all_values)
    all_groups = np.array(all_groups)

    tukey = pairwise_tukeyhsd(
        endog=all_values,   # the data
        groups=all_groups,  # group labels (folder names)
        alpha=alpha
    )

    print(f"\n=== Tukey HSD for {metric_col} (alpha = {alpha}) ===")
    print(tukey.summary())   # nice text table

    # Convert to DataFrame for easier viewing / export
    data = tukey._results_table.data
    header, rows = data[0], data[1:]
    tukey_df = pd.DataFrame(rows, columns=header)

    return tukey, tukey_df


In [10]:
#Goal 1

#get relevant folers
folder_names = ["Gyroid_8", "Gyroid_6", "Gyroid_4", "Solid_8", "Solid_6", "Solid_4", "Triangular_8", "Triangular_6", "Triangular_4"]
folder_addendum = ".is_tens_RawData"
parent_dir = "3DPrint"
folder_names_final = [os.path.join(parent_dir, name + folder_addendum) for name in folder_names]

#define lengths and section areas
L = 1 #in
A = 0.125*0.25 #in^2

#getting all the references
results_by_condition = {}

for i, folder_name in enumerate(folder_names_final):

    reference_tests = analyze_tensile_folder(
        folder_name,
        L, A
    )

    results_by_condition[folder_name] = reference_tests

#define metrics
metrics = [
    "UTS_Pa",              # ultimate tensile strength
    "TotalEnergy_Jperm3",  # energy absorbed
    "StrainAtFracture"     # elongation at break
]

#ANOVA
anova_results = []

for metric in metrics:
    F, p, groups = run_anova_on_metric(results_by_condition, metric)
    anova_results.append({
        "Metric": metric,
        "F_stat": F,
        "p_value": p
    })

print(anova_results)

#Shaprio
for metric in metrics:
    shapiro_df = run_shapiro_per_group(results_by_condition, metric)
    display(shapiro_df)

#levene's
for metric in metrics:
    run_levene_across_groups(results_by_condition, metric, center='median')

#Q-Q PLOT
# for metric in metrics:
#     qq_plots_per_group(results_by_condition, metric)

#Tukey
tukey_tables = {}

for metric in metrics:
    tukey_result, tukey_df = run_tukey_for_metric(results_by_condition, metric)
    tukey_tables[metric] = tukey_df
    display(tukey_df)



       Time  Extension         Load
1     0.000    0.00030     -2.64829
2     0.100    0.00764     43.94266
3     0.200    0.01751    104.44289
4     0.300    0.02691    132.00453
5     0.400    0.03581    182.97757
..      ...        ...          ...
384  37.884    3.16041  14899.19434
385  37.886    3.16064  13925.96191
386  37.888    3.16084  12905.07715
387  37.890    3.16104  11863.46484
388  37.892    3.16127  10824.25000

[388 rows x 3 columns]
       Time  Extension         Load
1     0.000    0.00000      0.93332
2     0.100    0.00403      2.96457
3     0.200    0.01341      7.14769
4     0.300    0.02283     80.13934
5     0.400    0.03210    169.43657
..      ...        ...          ...
364  35.826    2.98581  14494.72461
365  35.828    2.98601  13531.48340
366  35.830    2.98621  12524.41406
367  35.832    2.98641  11500.87891
368  35.834    2.98662  10481.74609

[368 rows x 3 columns]
       Time  Extension         Load
1     0.000    0.00000      8.55482
2     0.100    0

/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_82947/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_82947/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_82947/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_82947/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stre

,Condition,n,W_stat,p_value
0,3DPrint/Gyroid_8.is_tens_RawData,4,0.778583,0.069032
1,3DPrint/Gyroid_6.is_tens_RawData,4,0.792358,0.089206
2,3DPrint/Gyroid_4.is_tens_RawData,4,0.950580,0.719740
3,3DPrint/Solid_8.is_tens_RawData,4,0.827487,0.161342
4,3DPrint/Solid_6.is_tens_RawData,4,0.951657,0.726480
5,3DPrint/Solid_4.is_tens_RawData,4,0.824975,0.155074
6,3DPrint/Triangular_8.is_tens_RawData,4,0.911332,0.489485
7,3DPrint/Triangular_6.is_tens_RawData,4,0.999425,0.998424
8,3DPrint/Triangular_4.is_tens_RawData,4,0.934332,0.620089



=== Shapiro–Wilk normality test for TotalEnergy_Jperm3 ===
3DPrint/Gyroid_8.is_tens_RawData: n=4, W=0.9324, p=0.6087 → ✅ approx. normal (fail to reject H0)
3DPrint/Gyroid_6.is_tens_RawData: n=4, W=0.8827, p=0.3502 → ✅ approx. normal (fail to reject H0)
3DPrint/Gyroid_4.is_tens_RawData: n=4, W=0.9702, p=0.8427 → ✅ approx. normal (fail to reject H0)
3DPrint/Solid_8.is_tens_RawData: n=4, W=0.9245, p=0.5625 → ✅ approx. normal (fail to reject H0)
3DPrint/Solid_6.is_tens_RawData: n=4, W=0.9129, p=0.4978 → ✅ approx. normal (fail to reject H0)
3DPrint/Solid_4.is_tens_RawData: n=4, W=0.8315, p=0.1718 → ✅ approx. normal (fail to reject H0)
3DPrint/Triangular_8.is_tens_RawData: n=4, W=0.8570, p=0.2497 → ✅ approx. normal (fail to reject H0)
3DPrint/Triangular_6.is_tens_RawData: n=4, W=0.9842, p=0.9261 → ✅ approx. normal (fail to reject H0)
3DPrint/Triangular_4.is_tens_RawData: n=4, W=0.9535, p=0.7378 → ✅ approx. normal (fail to reject H0)


,Condition,n,W_stat,p_value
0,3DPrint/Gyroid_8.is_tens_RawData,4,0.932413,0.608655
1,3DPrint/Gyroid_6.is_tens_RawData,4,0.882681,0.350212
2,3DPrint/Gyroid_4.is_tens_RawData,4,0.970195,0.842693
3,3DPrint/Solid_8.is_tens_RawData,4,0.924512,0.562530
4,3DPrint/Solid_6.is_tens_RawData,4,0.912882,0.497804
5,3DPrint/Solid_4.is_tens_RawData,4,0.831538,0.171849
6,3DPrint/Triangular_8.is_tens_RawData,4,0.857010,0.249679
7,3DPrint/Triangular_6.is_tens_RawData,4,0.984180,0.926068
8,3DPrint/Triangular_4.is_tens_RawData,4,0.953459,0.737779


...

,group1,group2,meandiff,p-adj,lower,upper,reject
0,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Gyroid_6.is_tens_RawData,85602.4843,0.0000,77368.6342,93836.3344,True
1,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Gyroid_8.is_tens_RawData,132935.9062,0.0000,124702.0561,141169.7564,True
2,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_4.is_tens_RawData,355387.3749,0.0000,347153.5248,363621.2250,True
3,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_6.is_tens_RawData,363885.2500,0.0000,355651.3999,372119.1001,True
4,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,364401.8125,0.0000,356167.9624,372635.6626,True
5,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_4.is_tens_RawData,-7632.5234,0.0852,-15866.3736,601.3267,False
6,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_6.is_tens_RawData,59307.7108,0.0000,51073.8607,67541.5609,True
7,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_8.is_tens_RawData,143003.9218,0.0000,134770.0716,151237.7719,True
8,3DPrint/Gyroid_6.is_tens_RawData,3DPrint/Gyroid_8.is_tens_RawData,47333.4219,0.0000,39099.5718,55567.2720,True
9,3DPrint/Gyroid_6.is_tens_RawData,3DPrint/Solid_4.is_tens_RawData,269784.8906,0.0000,261551.0404,278018.7407,True



=== Tukey HSD for TotalEnergy_Jperm3 (alpha = 0.05) ===
                                       Multiple Comparison of Means - Tukey HSD, FWER=0.05                                       
               group1                               group2                   meandiff   p-adj      lower         upper     reject
---------------------------------------------------------------------------------------------------------------------------------
    3DPrint/Gyroid_4.is_tens_RawData     3DPrint/Gyroid_6.is_tens_RawData   303768.5553    0.0   153435.0842   454102.0264   True
    3DPrint/Gyroid_4.is_tens_RawData     3DPrint/Gyroid_8.is_tens_RawData   176213.5627 0.0128    25880.0916   326547.0338   True
    3DPrint/Gyroid_4.is_tens_RawData      3DPrint/Solid_4.is_tens_RawData   1490912.107    0.0  1340578.6359  1641245.5781   True
    3DPrint/Gyroid_4.is_tens_RawData      3DPrint/Solid_6.is_tens_RawData  1566966.6219    0.0  1416633.1508   1717300.093   True
    3DPrint/Gyroid_4.is_tens_RawD

,group1,group2,meandiff,p-adj,lower,upper,reject
0,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Gyroid_6.is_tens_RawData,3.037686e+05,0.0000,1.534351e+05,4.541020e+05,True
1,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Gyroid_8.is_tens_RawData,1.762136e+05,0.0128,2.588009e+04,3.265470e+05,True
2,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_4.is_tens_RawData,1.490912e+06,0.0000,1.340579e+06,1.641246e+06,True
3,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_6.is_tens_RawData,1.566967e+06,0.0000,1.416633e+06,1.717300e+06,True
4,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,1.569327e+06,0.0000,1.418994e+06,1.719661e+06,True
5,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_4.is_tens_RawData,-2.072107e+05,0.0022,-3.575441e+05,-5.687719e+04,True
6,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_6.is_tens_RawData,-3.981351e+04,0.9916,-1.901470e+05,1.105200e+05,False
7,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_8.is_tens_RawData,1.464482e+05,0.0606,-3.885271e+03,2.967817e+05,False
8,3DPrint/Gyroid_6.is_tens_RawData,3DPrint/Gyroid_8.is_tens_RawData,-1.275550e+05,0.1455,-2.778885e+05,2.277848e+04,False
9,3DPrint/Gyroid_6.is_tens_RawData,3DPrint/Solid_4.is_tens_RawData,1.187144e+06,0.0000,1.036810e+06,1.337477e+06,True



=== Tukey HSD for StrainAtFracture (alpha = 0.05) ===
                              Multiple Comparison of Means - Tukey HSD, FWER=0.05                               
               group1                               group2                meandiff p-adj   lower   upper  reject
----------------------------------------------------------------------------------------------------------------
    3DPrint/Gyroid_4.is_tens_RawData     3DPrint/Gyroid_6.is_tens_RawData   0.4523 0.0102  0.0753  0.8294   True
    3DPrint/Gyroid_4.is_tens_RawData     3DPrint/Gyroid_8.is_tens_RawData    -0.02    1.0 -0.3971   0.357  False
    3DPrint/Gyroid_4.is_tens_RawData      3DPrint/Solid_4.is_tens_RawData   1.4803    0.0  1.1032  1.8573   True
    3DPrint/Gyroid_4.is_tens_RawData      3DPrint/Solid_6.is_tens_RawData   1.4349    0.0  1.0579   1.812   True
    3DPrint/Gyroid_4.is_tens_RawData      3DPrint/Solid_8.is_tens_RawData   1.4698    0.0  1.0927  1.8468   True
    3DPrint/Gyroid_4.is_tens_RawData 3DPr

,group1,group2,meandiff,p-adj,lower,upper,reject
0,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Gyroid_6.is_tens_RawData,0.4523,0.0102,0.0753,0.8294,True
1,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Gyroid_8.is_tens_RawData,-0.0200,1.0000,-0.3971,0.3570,False
2,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_4.is_tens_RawData,1.4803,0.0000,1.1032,1.8573,True
3,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_6.is_tens_RawData,1.4349,0.0000,1.0579,1.8120,True
4,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,1.4698,0.0000,1.0927,1.8468,True
5,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_4.is_tens_RawData,-0.7056,0.0000,-1.0826,-0.3286,True
6,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_6.is_tens_RawData,-0.4218,0.0198,-0.7989,-0.0448,True
7,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_8.is_tens_RawData,-0.2107,0.6321,-0.5877,0.1664,False
8,3DPrint/Gyroid_6.is_tens_RawData,3DPrint/Gyroid_8.is_tens_RawData,-0.4723,0.0066,-0.8494,-0.0953,True
9,3DPrint/Gyroid_6.is_tens_RawData,3DPrint/Solid_4.is_tens_RawData,1.0279,0.0000,0.6509,1.4050,True


#GOAL 1
